In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from collections import deque
import numpy as np
import multiprocessing
import time
from numpy import array
import pandas as pd
import dask.dataframe as dd
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(1)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
samplesize = 5

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls '/content/drive/My Drive/QueenslandDataset/'

 Case01.csv   Case08.csv   Case15.csv   Case22.csv   Case29.csv
 Case02.csv   Case09.csv   Case16.csv   Case23.csv   Case30.csv
 Case03.csv   Case10.csv   Case17.csv   Case24.csv   Case31.csv
 Case04.csv   Case11.csv   Case18.csv   Case25.csv   Case32.csv
 Case05.csv   Case12.csv   Case19.csv   Case26.csv  'Processed Data'
 Case06.csv   Case13.csv   Case20.csv   Case27.csv   uqvitalsignsdata
 Case07.csv   Case14.csv   Case21.csv   Case28.csv


In [2]:
ecg = []
ppg = []
ppg_offset = []
HR = []
sys = []
dias = []
offset_list = []
column_list = []
for i in range(1,31):
    case = pd.read_csv(f'/content/drive/My Drive/QueenslandDataset/Case{i:02d}.csv')
    case_copy = case.copy()
    print(case.shape)
    start = 0
    step = 100
    end = 1000
    while True:        
        flag = False
        offset = random.randint(-50,50)
        offset_list.append(offset)
        if(start+end>case.shape[0]):
            flag = True
        if start+offset<0:
          index = 0
        else:
          index = start+offset
        chunk = case[start:start+end]
        # ppg_row =chunk['Pleth'].to_numpy()
        # # ppg.append(ppg_row)
        # ppg_offset_row = case_copy[index:index+end]['Pleth'].to_numpy()
        # # print(start, end, offset )
        # # print("ppg",ppg_row)
        # # print("ppg_offset",ppg_offset_row)
        # # print("offset", offset)
        # ppg_offset.append(ppg_offset_row)
        # ecg_row =chunk['ECG'].to_numpy()
        # ecg.append(ecg_row)
        # hr_row = chunk['HR'].to_numpy()
        # HR.append(hr_row)
        # sys_row = chunk['SYS'].to_numpy()
        # sys.append(sys_row)
        dias_row = chunk['Dias'].to_numpy()
        dias.append(dias_row)
        # ecg.append(case['ECG'].to_numpy().tolist())
        # HR.append()
        # sys.append()
        # dias.append()
        # ppg_offset.append()
        start += step
        if flag:
            break
        # break
    # break

# ppg_df = pd.DataFrame(ppg)

(729600, 6)
(101376, 6)
(1498522, 6)
(876578, 6)
(1192551, 6)
(632116, 6)
(410034, 6)
(349389, 6)
(654832, 6)
(258457, 6)
(540660, 6)
(1771628, 6)
(594536, 6)
(533215, 6)
(158958, 6)
(1148642, 6)
(122266, 6)
(69427, 6)
(139674, 6)
(675123, 6)
(458547, 6)
(977954, 6)
(167733, 6)
(372308, 6)
(633856, 6)
(997822, 6)
(1538143, 6)
(446111, 6)
(730761, 6)
(368272, 6)


In [15]:
ppg

[array([0.3685 , 0.36801, 0.36728, 0.3663 , 0.36508, 0.36215, 0.35995,
        0.35726, 0.35458, 0.35043]),
 array([0.36728, 0.3663 , 0.36508, 0.36215, 0.35995, 0.35726, 0.35458,
        0.35043, 0.34945, 0.34872]),
 array([0.36508, 0.36215, 0.35995, 0.35726, 0.35458, 0.35043, 0.34945,
        0.34872, 0.34847, 0.34994]),
 array([0.35995, 0.35726, 0.35458, 0.35043, 0.34945, 0.34872, 0.34847,
        0.34994, 0.35189, 0.35458]),
 array([0.35458, 0.35043, 0.34945, 0.34872, 0.34847, 0.34994, 0.35189,
        0.35458, 0.35775, 0.36703]),
 array([0.34945, 0.34872, 0.34847, 0.34994, 0.35189, 0.35458, 0.35775,
        0.36703, 0.37289, 0.37998]),
 array([0.34847, 0.34994, 0.35189, 0.35458, 0.35775, 0.36703, 0.37289,
        0.37998, 0.38706, 0.40488])]

In [16]:
ecg

[array([-0.305, -0.31 , -0.325, -0.325, -0.33 , -0.355, -0.385, -0.395,
        -0.395, -0.38 ]),
 array([-0.325, -0.325, -0.33 , -0.355, -0.385, -0.395, -0.395, -0.38 ,
        -0.425, -0.45 ]),
 array([-0.33 , -0.355, -0.385, -0.395, -0.395, -0.38 , -0.425, -0.45 ,
        -0.435, -0.46 ]),
 array([-0.385, -0.395, -0.395, -0.38 , -0.425, -0.45 , -0.435, -0.46 ,
        -0.46 , -0.37 ]),
 array([-0.395, -0.38 , -0.425, -0.45 , -0.435, -0.46 , -0.46 , -0.37 ,
        -0.3  , -0.27 ]),
 array([-0.425, -0.45 , -0.435, -0.46 , -0.46 , -0.37 , -0.3  , -0.27 ,
        -0.235, -0.23 ]),
 array([-0.435, -0.46 , -0.46 , -0.37 , -0.3  , -0.27 , -0.235, -0.23 ,
        -0.24 , -0.185])]

In [17]:
ppg_offset

[array([], dtype=float64),
 array([0.35995, 0.35726, 0.35458, 0.35043, 0.34945, 0.34872, 0.34847,
        0.34994, 0.35189, 0.35458]),
 array([0.36801, 0.36728, 0.3663 , 0.36508, 0.36215, 0.35995, 0.35726,
        0.35458, 0.35043, 0.34945]),
 array([0.35458, 0.35043, 0.34945, 0.34872, 0.34847, 0.34994, 0.35189,
        0.35458, 0.35775, 0.36703]),
 array([0.35458, 0.35043, 0.34945, 0.34872, 0.34847, 0.34994, 0.35189,
        0.35458, 0.35775, 0.36703]),
 array([0.34847, 0.34994, 0.35189, 0.35458, 0.35775, 0.36703, 0.37289,
        0.37998, 0.38706, 0.40488]),
 array([0.34945, 0.34872, 0.34847, 0.34994, 0.35189, 0.35458, 0.35775,
        0.36703, 0.37289, 0.37998])]

In [18]:
offset

-2

In [30]:
ppg_df = pd.DataFrame(ppg)

In [31]:
ppg_df.to_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/ppg.csv')

In [3]:
ppg_offset_df = pd.DataFrame(ppg_offset)

In [4]:
ppg_offset_df.to_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/ppg_offset.csv')

In [5]:
offset_df = pd.DataFrame(offset_list)

In [6]:
offset_df.to_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/offset.csv')

In [3]:
ecg_df = pd.DataFrame(ecg)

In [4]:
ecg_df.to_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/ecg.csv')

In [6]:
hr_df = pd.DataFrame(HR)

In [7]:
hr_df.to_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/hr.csv')

In [4]:
sys_df = pd.DataFrame(sys)

In [5]:
sys_df.to_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/sys.csv')

In [3]:
dias_df = pd.DataFrame(dias)

In [4]:
dias_df.to_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/dias.csv')

In [ ]:
dias

[array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0.,  0.,  0., ..., 63., 63., 63.]),
 array([ 0

In [ ]:
ppg_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0.36850,0.36801,0.36728,0.36630,0.36508,0.36215,0.35995,0.35726,0.35458,0.35043,0.34945,0.34872,0.34847,0.34994,0.35189,0.35458,0.35775,0.36703,0.37289,0.37998,0.38706,0.40488,0.41709,0.43101,0.44664,0.47961,0.49548,0.51038,0.52357,0.54383,0.55067,0.55067,0.56801,0.59365,0.61099,0.61099,0.61099,0.61099,0.61099,0.78071,...,0.52894,0.52991,0.53040,0.53065,0.53114,0.53162,0.53211,0.53236,0.53260,0.53260,0.53260,0.53236,0.53114,0.53065,0.53040,0.53016,0.53016,0.53016,0.53016,0.53016,0.53016,0.53016,0.53016,0.52967,0.52869,0.52845,0.52845,0.52869,0.52918,0.53114,0.53260,0.53358,0.53455,0.53578,0.53602,0.53602,0.53602,0.53602,0.53602,0.53602
1,0.40024,0.40464,0.40659,0.40708,0.40562,0.39585,0.38730,0.37705,0.36557,0.34408,0.33626,0.32869,0.31770,0.31502,0.31355,0.31331,0.31404,0.31673,0.31844,0.32063,0.32527,0.32772,0.33040,0.33333,0.33993,0.34359,0.34701,0.35043,0.35385,0.36117,0.36484,0.37411,0.37411,0.38584,0.39219,0.39219,0.39219,0.39219,0.41978,0.43028,...,0.55092,0.55385,0.55507,0.55629,0.55824,0.55922,0.55995,0.56068,0.56190,0.56264,0.56313,0.56288,0.56166,0.56068,0.55946,0.55849,0.55751,0.55556,0.55482,0.55409,0.55409,0.55507,0.55580,0.55702,0.56117,0.56337,0.56532,0.56679,0.56947,0.57070,0.57167,0.57289,0.57436,0.57778,0.57949,0.58120,0.58291,0.58486,0.58486,0.58486
2,0.42808,0.42173,0.41563,0.41001,0.40049,0.39658,0.39292,0.38950,0.38657,0.38169,0.37998,0.37778,0.37534,0.37094,0.36923,0.36801,0.36752,0.36850,0.36947,0.37094,0.37387,0.37485,0.37534,0.37558,0.37534,0.37534,0.37558,0.37607,0.37680,0.37875,0.37998,0.37998,0.38242,0.38437,0.38535,0.38535,0.38535,0.38535,0.38559,0.38584,...,0.52967,0.52894,0.52747,0.52576,0.51941,0.51380,0.50623,0.49768,0.47350,0.45812,0.44151,0.42369,0.40635,0.37509,0.35995,0.34554,0.33370,0.32430,0.31722,0.31038,0.30549,0.30110,0.30037,0.29988,0.29963,0.29963,0.29963,0.29963,0.30012,0.30183,0.30379,0.30647,0.31136,0.31502,0.31893,0.32283,0.32698,0.33358,0.33651,0.34066
3,0.48523,0.48303,0.48205,0.48107,0.48010,0.47766,0.47619,0.47473,0.47302,0.46960,0.46740,0.46471,0.45934,0.45641,0.45372,0.45128,0.44908,0.44542,0.44371,0.44274,0.44249,0.44396,0.44591,0.44908,0.45275,0.46056,0.46422,0.46691,0.46935,0.46935,0.46789,0.46520,0.46520,0.45690,0.44640,0.44640,0.44640,0.44640,0.42442,0.42442,...,0.33333,0.33382,0.33504,0.33675,0.33822,0.33846,0.33748,0.33578,0.33455,0.33358,0.33333,0.33333,0.33333,0.33333,0.33333,0.33382,0.33675,0.33871,0.34066,0.34286,0.34554,0.35092,0.35311,0.35531,0.35751,0.36044,0.36215,0.36410,0.36801,0.36899,0.36947,0.36972,0.36899,0.36557,0.36361,0.36166,0.35971,0.35287,0.34994,0.34750
4,0.50379,0.50427,0.50501,0.50623,0.50769,0.51111,0.51282,0.51453,0.51575,0.51673,0.51697,0.51746,0.51893,0.52015,0.52161,0.52381,0.52650,0.53284,0.53651,0.53968,0.54237,0.54945,0.55311,0.55678,0.55995,0.56557,0.56752,0.56899,0.56972,0.56899,0.56777,0.56777,0.56337,0.55751,0.55043,0.55043,0.55043,0.55043,0.55043,0.53065,...,0.48889,0.48523,0.48132,0.47741,0.47082,0.46886,0.46740,0.46642,0.46520,0.46545,0.46642,0.46764,0.46911,0.47155,0.47253,0.47326,0.47424,0.47619,0.47692,0.47766,0.47863,0.47937,0.47888,0.47766,0.47302,0.47033,0.46764,0.46520,0.46300,0.45836,0.45665,0.45568,0.45543,0.45641,0.45739,0.45910,0.46154,0.46520,0.46642,0.46740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191233,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,

In [ ]:
ppg_offset_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0.35189,0.35458,0.35775,0.36703,0.37289,0.37998,0.38706,0.40488,0.41709,0.43101,0.44664,0.47961,0.49548,0.51038,0.52357,0.54383,0.55067,0.55067,0.56801,0.59365,0.61099,0.61099,0.61099,0.61099,0.61099,0.78071,0.80781,0.86105,0.89182,0.92332,0.97705,0.98852,0.99414,0.99707,0.99853,0.99951,0.99976,0.99976,0.99976,0.99976,...,0.53040,0.53016,0.53016,0.53016,0.53016,0.53016,0.53016,0.53016,0.53016,0.52967,0.52869,0.52845,0.52845,0.52869,0.52918,0.53114,0.53260,0.53358,0.53455,0.53578,0.53602,0.53602,0.53602,0.53602,0.53602,0.53602,0.53602,0.53675,0.53748,0.53822,0.53846,0.53895,0.53895,0.53895,0.53895,0.53895,0.53895,0.53895,0.53895,0.53895
1,0.29475,0.32088,0.33602,0.35629,0.36484,0.37387,0.38242,0.39463,0.39756,0.39878,0.39805,0.39756,0.39780,0.39878,0.40024,0.40464,0.40659,0.40708,0.40562,0.39585,0.38730,0.37705,0.36557,0.34408,0.33626,0.32869,0.31770,0.31502,0.31355,0.31331,0.31404,0.31673,0.31844,0.32063,0.32527,0.32772,0.33040,0.33333,0.33993,0.34359,...,0.54017,0.54042,0.54042,0.54042,0.54066,0.54090,0.54139,0.54261,0.54359,0.54432,0.54530,0.54676,0.54774,0.54921,0.55092,0.55385,0.55507,0.55629,0.55824,0.55922,0.55995,0.56068,0.56190,0.56264,0.56313,0.56288,0.56166,0.56068,0.55946,0.55849,0.55751,0.55556,0.55482,0.55409,0.55409,0.55507,0.55580,0.55702,0.56117,0.56337
2,0.61661,0.61661,0.61661,0.61661,0.61661,0.62247,0.62369,0.62369,0.62247,0.61538,0.61001,0.60415,0.59780,0.58364,0.57582,0.56777,0.55922,0.54115,0.53187,0.52308,0.50696,0.50061,0.49524,0.49011,0.48571,0.47790,0.47399,0.46984,0.46496,0.45299,0.44615,0.44005,0.42808,0.42173,0.41563,0.41001,0.40049,0.39658,0.39292,0.38950,...,0.62784,0.61416,0.60562,0.60562,0.58388,0.58388,0.57411,0.57411,0.57411,0.57411,0.57411,0.54432,0.53993,0.53529,0.53065,0.52747,0.52283,0.52234,0.52430,0.52698,0.52918,0.53553,0.53773,0.53895,0.53895,0.53431,0.53187,0.53065,0.52991,0.52967,0.52967,0.52967,0.52967,0.52894,0.52747,0.52576,0.51941,0.51380,0.50623,0.49768
3,0.44591,0.44908,0.45275,0.46056,0.46422,0.46691,0.46935,0.46935,0.46789,0.46520,0.46520,0.45690,0.44640,0.44640,0.44640,0.44640,0.42442,0.42442,0.42418,0.42589,0.42784,0.43053,0.43346,0.43346,0.43346,0.44591,0.44957,0.45031,0.44982,0.44982,0.44786,0.44689,0.44640,0.44615,0.44689,0.45055,0.45348,0.45958,0.45958,0.46593,...,0.35311,0.35531,0.35751,0.36044,0.36215,0.36410,0.36801,0.36899,0.36947,0.36972,0.36899,0.36557,0.36361,0.36166,0.35971,0.35287,0.34994,0.34750,0.34505,0.34164,0.34212,0.34359,0.34408,0.34408,0.34457,0.34579,0.34750,0.35189,0.35531,0.35897,0.36850,0.37314,0.39731,0.43101,0.44933,0.46691,0.47179,0.47619,0.48034,0.48694
4,0.47497,0.47790,0.48083,0.48400,0.49035,0.49353,0.49621,0.49866,0.50208,0.50305,0.50354,0.50379,0.50427,0.50501,0.50623,0.50769,0.51111,0.51282,0.51453,0.51575,0.51673,0.51697,0.51746,0.51893,0.52015,0.52161,0.52381,0.52650,0.53284,0.53651,0.53968,0.54237,0.54945,0.55311,0.55678,0.55995,0.56557,0.56752,0.56899,0.56972,...,0.51966,0.52088,0.52112,0.52063,0.51697,0.51404,0.51038,0.50672,0.50061,0.49817,0.49231,0.48889,0.48523,0.48132,0.47741,0.47082,0.46886,0.46740,0.46642,0.46520,0.46545,0.46642,0.46764,0.46911,0.47155,0.47253,0.47326,0.47424,0.47619,0.47692,0.47766,0.47863,0.47937,0.47888,0.47766,0.47302,0.47033,0.46764,0.46520,0.46300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7283,0.68718,0.66764,0.63834,0.62882,0.62222,0.61832,0.61636,0.61734,0.62027,0.62418,0.60024,0.54554,0.53846,0.53553,0.53358,0.52918,0.52601,0.52210,0.51746,0.50720,0.